In [3]:
import numpy as np
import gzip
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

def load_emnist_images(filename):
    with gzip.open(filename, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=16)
    data = data.reshape(-1, 28, 28)
    return data

def load_emnist_labels(filename):
    with gzip.open(filename, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=8)
    return data

# Google Drive folder path
folder_path = '/content/drive/My Drive/A Final Project/Data'

# Load training images and labels
x_train = load_emnist_images(os.path.join(folder_path, 'emnist-letters-train-images-idx3-ubyte.gz'))
y_train = load_emnist_labels(os.path.join(folder_path, 'emnist-letters-train-labels-idx1-ubyte.gz'))

# Load test images and labels
x_test = load_emnist_images(os.path.join(folder_path, 'emnist-letters-test-images-idx3-ubyte.gz'))
y_test = load_emnist_labels(os.path.join(folder_path, 'emnist-letters-test-labels-idx1-ubyte.gz'))

# Load mapping
with open(os.path.join(folder_path, 'emnist-letters-mapping.txt'), 'r') as f:
    mapping = f.readlines()
mapping = [line.split() for line in mapping]
mapping = {int(line[0]): chr(int(line[1])) for line in mapping}

# Convert labels to characters
y_train = np.array([mapping[label] for label in y_train])
y_test = np.array([mapping[label] for label in y_test])

Mounted at /content/drive


In [4]:
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

from sklearn.preprocessing import OneHotEncoder

# One-hot encode the labels
encoder = OneHotEncoder(sparse=False)
y_train_onehot = encoder.fit_transform(y_train.reshape(-1, 1))
y_test_onehot = encoder.transform(y_test.reshape(-1, 1))

# Verify the shape and unique classes
print(f"Shape of y_train_onehot: {y_train_onehot.shape}")
print(f"Unique classes in y_train: {np.unique(y_train)}")

Shape of y_train_onehot: (124800, 26)
Unique classes in y_train: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [5]:
# Model definition and compilation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.utils import set_random_seed
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

# This is needed for replicability
set_random_seed(555)
total_num_classes = 26  # For the 26 alphabet letters

In [6]:
model4 = Sequential()
model4.add(Conv2D(64, kernel_size=(3, 3), input_shape=(28, 28, 1), activation="sigmoid"))  # Increased number of filters
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Conv2D(128, kernel_size=(2, 2), activation="sigmoid"))  # Increased number of filters
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Conv2D(256, kernel_size=(2, 2), activation="sigmoid"))  # Increased number of filters
model4.add(MaxPooling2D(pool_size=(2, 2)))
model4.add(Flatten())
model4.add(Dense(128, activation="sigmoid"))
model4.add(Dense(total_num_classes, activation="softmax"))

# Configure optimizer and loss
optim = Adam(learning_rate=0.001)
loss_fn = CategoricalCrossentropy()

model4.compile(loss=loss_fn, optimizer=optim, metrics=["accuracy"])

batch_size = 128
epochs = 15

model4.fit(x_train, y_train_onehot, batch_size=batch_size, epochs=epochs)

Epoch 1/15
975/975 [==============================] - 278s 283ms/step - loss: 1.1995 - accuracy: 0.6611
Epoch 2/15
975/975 [==============================] - 273s 280ms/step - loss: 0.3570 - accuracy: 0.8914
Epoch 3/15
581/975 [================>.............] - ETA: 1:51 - loss: 0.2762 - accuracy: 0.9125

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score

# Create a mapping dictionary
mapping = {i: chr(i + 65) for i in range(26)}

predictions4 = model4.predict(x_test)
predicted_class_indices4 = predictions4.argmax(axis=1)
true_class_indices = np.argmax(y_test_onehot, axis=1)

# Get the class labels for predictions and true labels
predicted_class_labels4 = [mapping[i] for i in predicted_class_indices4]
true_class_labels = [mapping[i] for i in true_class_indices]

# Compute accuracy
acc4 = accuracy_score(true_class_labels, predicted_class_labels4)

print(f"Testing accuracy: Model 4 = {acc4}")

In [ ]:
import cv2
from matplotlib import pyplot as plt
import os

# Path to the folder containing the images
folder_path = '/content/drive/My Drive/A Final Project/Letters'

# Loop through each image file from '1.png' to '19.png'
for i in range(1, 20):
    file = os.path.join(folder_path, f"{i}.png")

    # Load the image
    test_image = cv2.imread(file, cv2.IMREAD_GRAYSCALE)

    # Check if the image is loaded successfully
    if test_image is None:
        print(f"Error: Unable to load the image {file}.")
    else:
        # Preprocess the image
        img_resized = cv2.resize(test_image, (28, 28), interpolation=cv2.INTER_LINEAR)
        img_resized = cv2.bitwise_not(img_resized)

        # Preprocess the image for prediction
        img_resized_new = np.expand_dims(img_resized, -1)
        img_resized_new = np.expand_dims(img_resized_new, 0)

        # Make predictions
        predicted_scores = model4.predict(img_resized_new)

        # Get the predicted letter index
        predicted_letter_index = np.argmax(predicted_scores)  # Get the index of the highest score

        # Convert the predicted index to the corresponding letter
        predicted_letter = chr(predicted_letter_index + ord('A'))  # Assuming your model predicts letters from 'A' to 'Z'

        #print({predicted_letter}, end="")

        print(f"Predicted letter for {file}: {predicted_letter}")
